In [9]:
import boto3
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'ryan-greiner-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading CSV file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [6]:
heart = heart.dropna()
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2)

scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)
X_train = scaler.fit_transform(X_train)

In [8]:
logit_md = LogisticRegression().fit(X_train, Y_train)
logit_pred = logit_md.predict_proba(X_test)[:, 1]
logit_label = np.where(logit_pred < .1, 0, 1)
logit_recall = recall_score(Y_test, logit_label)
print('The recall of the logistic model is', logit_recall)

The recall of the logistic model is 0.7931034482758621


In [10]:
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)
rf_pred = rf_md.predict_proba(X_test)[:, 1]
rf_label = np.where(rf_pred < .1, 0, 1)
rf_recall = recall_score(Y_test, rf_label)
print('The recall of the random forest model is', rf_recall)

The recall of the random forest model is 0.8275862068965517


In [ ]:
# from the results above, the random forest model is prefered since it has a higher recall score